
Celem projektu jest analiza danych dotyczących udaru mózgu, znajdujących się w pliku `healthcare-dataset-stroke-data.csv`. Wyniki należy przedstawić w formie  raportu (RMarkdown, Jupyter, etc.) zawierającego kod programu jak i opisy dokonywanych decyzji. Analiza powinna zawierać co najmniej następujące punkty:

1. Czyszczenie danych (usuwanie/inputacja braków danych, naprawa błędów, transformacje danych, rozwiązanie problemu wartości odstających)
2. Eksploracyjna analiza danych
3. Zamodelowanie zmiennej `stroke` na podstawie pozostałych zmiennych. Minimum 3 modele.
4. Ewaluacja na zbiorze testowym (wybór modelu i metryk z uzasadnieniem)

Możliwe jest rozwiązaniem w języku R lub Python. Rozwiązania wysłać do 24.01.2022 na adres mailowy: michalmaj116@gmail.com z tytułem "studia podyplomowe"


# 1. Importy

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, roc_curve, auc

from imblearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import BaggingClassifier
from imblearn.over_sampling import SMOTE
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay

# 2. Wczytanie danych

In [2]:
data_stroke = pd.read_csv("healthcare-dataset-stroke-data.csv", comment="#", sep=',')
data_stroke.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


KeyError: "['stroke'] not found in axis"

# 3. Zrozumienie danych
##### Dane dotyczące udaru mózgu.
**id** -numer pacjenta

**gender** - płeć

**age** - wiek w latach

**hypertension** - nadciśnienie (0/1) 
 
**heart_disease** - choroba serca (0/1)

**ever_married** - czy pacjent był zonaty/zamężny (yes/no)

**Residence_type** - miejsce zamieszkania - urban - miejski, rural - wiejski

**work_type** - typ pracy - 5 rodzajów - self-employed(samozatrudniony), Govt_job - praca rządowa

**avg_glucose_level** -średni poziom glukozy, nie ma jednostek i skali??? (wg. 	eAG (mg/dL) - może być do 300
    Normal range: less than 114 mg/dL
    Prediabetes range: greater than 114 mg/dL and less than 140 mg/dL
    Diabetes range: greater than 140 mg/dL
    
**bmi** - bmi

**smoking_status** -formerly smoked - wcześniej palący

**stroke** - zmienna celu

In [ ]:
type(data_stroke)
data_stroke.shape
data_stroke.columns
data_stroke.info()

In [ ]:
#INFORMACJE O DANYCH
#rózne typy danych - trzeba pozmieniać
#BMI - barkuje 201 kolumn - ?? - trzeba zrobić coś na NaNami
#zmienić hypertension, heart_disease, do ever_maried, work_type, residence_type, smiking_status zobiektowych na kategoryczne


In [ ]:
#pozmieniam zmienne objekt na category - jak to zrobić w jednej linijce???
data_stroke.gender = data_stroke.gender.astype('category')
data_stroke.hypertension = data_stroke.hypertension.astype('category')
data_stroke.heart_disease = data_stroke.heart_disease.astype('category')
data_stroke.ever_married = data_stroke.ever_married.astype('category')
data_stroke.work_type = data_stroke.work_type.astype('category')
data_stroke.Residence_type = data_stroke.Residence_type.astype('category')
data_stroke.smoking_status = data_stroke.smoking_status.astype('category')
data_stroke.stroke = data_stroke.stroke.astype('category')

In [ ]:
#usunięcie kolumny ID - nic nie wnosi
data_stroke.drop('id', axis=1, inplace=True)

In [ ]:
#szukanie braków danych
data_stroke_nan_index = data_stroke.index[data_stroke.isnull().any(axis=1)]
data_stroke.iloc[data_stroke_nan_index]

In [ ]:
#usuwanie/imputacja braków danych - wszystkich obserwacji 5110 (100%), więc 201 to będzie niecałe 4%, 
#zacznę od usunięcia i zobaczę co dalej będzie
data_stroke.dropna(inplace=True)
data_stroke.info()

In [ ]:
#imputer = KNNImputer(n_neighbors=5)
#data_stroke_imputed = imputer.fit_transform(data_stroke)

#imp = SimpleImputer(strategy='mean')
#data_stroke_imp = imp.fit_transform(data_stroke)

In [ ]:
#poznaje kolejne kolumny nietypowe
data_stroke.groupby('gender').count()
#gender ma male, female ( w miarę równo rozłożone) i kolumne inne(1 odczyt)

In [ ]:
data_stroke.drop(data_stroke[data_stroke['gender']=='Other'].index, inplace=True) ###tu usunąc tego other!!!
data_stroke.groupby('gender').count()

In [ ]:
data_stroke.groupby('ever_married').count()
#kolumny yes, no(trochę nierówno 1727:3353)

In [ ]:
data_stroke.groupby('work_type').count()
#5 rodzajów - govt job - praca rządowa,

In [ ]:
data_stroke.groupby('Residence_type').count()
#tylko miejskie i wiejskie(równo ułożone)

In [ ]:
data_stroke.groupby('smoking_status').count()
#4 rodzaje - w tym jeden - unknown

In [ ]:
data_stroke.groupby('stroke').count()
#zmienna celu
#tu mamy dwa rodzaje - duża nierówność prawie 5000:250

In [ ]:
#Przy klasyfikacji jednym z najwazniejszych pytań podczas eksploracyjnej analizy danych 
#jest zbalansowanie zmiennej zaleznej a raczej jego brak. 
#W przypadku niezbalansowanych danych dużo trudniej jest zbudować dobrze działający klasyfikator.

counts = data_stroke['stroke'].value_counts()
percentages = counts / counts.sum() * 100

plt.bar(percentages.index, percentages.values) # zmienić na yes/no poziomą wartość
plt.title('Stroke Column')
plt.xlabel('Stroke')
plt.ylabel('Percentage')
plt.show()
#zmienna niezbilansowana 

In [ ]:
#badam zmienne numeryczne
data_stroke.describe()

# 4. Outliery 
wiek i glukoza wyglądają znośnie, BMI ma dziwne wyniki

poniżej 16 - wygłodzenie
16 - 16.99 - wychudzenie
17 - 18.49 - niedowagę
18.5 - 24.99 - wagę prawidłową
25.0 - 29.9 - nadwagę
30.0 - 34.99 - I stopień otyłości
35.0 - 39.99 - II stopień otyłości
powyżej 40.0 - otyłość skrajną


In [ ]:
#Szukam outlierów metodą z_scores
data_stroke_imputed = pd.DataFrame(data_stroke, columns=['age', 'avg_glucose_level', 'bmi'])

means = np.mean(data_stroke_imputed, axis=0)
sds = np.std(data_stroke_imputed, axis=0)
z_scores = np.abs(data_stroke_imputed - means) / sds
outliers = np.where(z_scores > 3)

print(outliers)

In [ ]:
data_stroke_imputed.iloc[outliers[0][outliers[1] == 2]]
data_stroke = data_stroke[data_stroke["bmi"] <= 52] #powyżej 60 to chyba niemożliwe 

In [ ]:
num_cols = ['age', 'avg_glucose_level', 'bmi']

for col in num_cols:
    fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(10, 5))
    sns.histplot(data=data_stroke, x=col, hue='stroke', kde=True, ax=ax1)
    sns.boxplot(data=data_stroke, x='stroke', y=col, ax=ax2)
    fig.suptitle(f'{col} by stroke', fontsize=14)
    plt.show()

In [ ]:
data_stroke

In [ ]:
object_columns = data_stroke.select_dtypes(include='object').columns.drop('stroke')

for col in object_columns:
    fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(16, 6))
    
    adult.groupby(col)['stroke'].value_counts(normalize=False).unstack().plot(kind='bar', stacked=True, ax=ax[0])
    ax[0].set_title(col + ' (Non-Normalized)')
    ax[0].set_xlabel(col)
    ax[0].set_ylabel('Count')

    adult.groupby(col)['stroke'].value_counts(normalize=True).unstack().plot(kind='bar', stacked=True, ax=ax[1])
    ax[1].set_title(col + ' (Normalized)')
    ax[1].set_xlabel(col)
    ax[1].set_ylabel('Procent')

    plt.show()

# 5. Korelacja

In [ ]:
#Sprawdzam ewentualną korelację pomiędzy zmiennymi  numerycznymi
plt.figure(figsize = (3,3))
plt.title("Correlation between different features of the dataset", fontsize = 18, fontweight = 'bold')
sns.heatmap(data_stroke.corr(numeric_only = True), cmap = 'Greens_r', annot = True)
plt.xticks(fontsize=12, rotation = 90)
plt.yticks(fontsize=12, rotation = 90)
plt.legend(fontsize=12)

#korelacji nie ma
#czy da się zbadać korelcję z kategorycznymi???

Nie widzę większej korelacji pomiędzy zmiennymi numerycznymi.


# 6. Wykresy

In [ ]:
def generate_plots_and_tables_by_tagret(df: pd.DataFrame, target_column: str = "DefFlag",
                                        numeric_cols: list = [], categorical_cols: list = []) -> None:
    """
    Generate plots of the 'target_column' column with other columns in a pandas dataframe.
    If a column is a string or factor, a bar plot with grouping by 'target_column' will be created.
    If a column is continuous, a histogram, boxplot, and scatter plot with grouping by 'target_column' will be created.
    """
    for column in categorical_cols:
        print(column + ":")
        
        plt.figure(figsize=(8, 6))
            
        counts = df.groupby([column, target_column], group_keys=True).size()
        print(counts)
        counts_norm = counts.groupby(level=0, group_keys=False).apply(lambda x: 100 * x / x.sum())
        counts_norm = counts_norm.reset_index(name='percent')
        sns.barplot(x=column, y='percent', hue=target_column, data=counts_norm)
        plt.title(f"'stroke' vs '{column}'")
        plt.show()
            
    for column in numeric_cols:
        
        plt.figure(figsize=(18, 6))
            
        plt.subplot(1, 2, 1)
        sns.histplot(x=column, hue=target_column, data=df, kde=True)
        plt.title(f"'stroke' vs '{column}' (Histogram)")
            
        plt.subplot(1, 2, 2)
        sns.boxplot(x=target_column, y=column, data=df)
        plt.title(f"'stroke' vs '{column}' (Boxplot)")
            
        plt.tight_layout()
        plt.show()

In [ ]:
num_cols = ['age', 'avg_glucose_level', 'bmi']
cat_cols = ['gender', 'hypertension', 'heart_disease', 'ever_married', 'work_type', 'Residence_type', 'smoking_status']
generate_plots_and_tables_by_tagret(data_stroke, 'stroke', num_cols, cat_cols)

In [ ]:
object_columns = data_stroke.select_dtypes(include='object').columns.drop('stroke')

for col in object_columns:
    fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(16, 6))
    
    adult.groupby(col)['stroke'].value_counts(normalize=False).unstack().plot(kind='bar', stacked=True, ax=ax[0])
    ax[0].set_title(col + ' (Non-Normalized)')
    ax[0].set_xlabel(col)
    ax[0].set_ylabel('Count')

    adult.groupby(col)['stroke'].value_counts(normalize=True).unstack().plot(kind='bar', stacked=True, ax=ax[1])
    ax[1].set_title(col + ' (Normalized)')
    ax[1].set_xlabel(col)
    ax[1].set_ylabel('Procent')

    plt.show()

Widzę pewną zależność pomiędzy:

-nadciśnieniem, a udarami

-chorobami serca a udarami

-wydaje mi się, że proporcja samozatrudnienia a udarów jest wyższa


# 7. Tworzenie modeli


In [ ]:
X, y = data_stroke.drop(columns=['stroke'], axis=1), data_stroke['stroke']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

cat_ix = X.select_dtypes(include=['object', 'bool']).columns
num_ix = X.select_dtypes(include=['int64', 'float64']).columns

In [ ]:
names = ['CART', 'BAG', 'RF', 'KN', 'LR', 'GBM']
models = [
    DecisionTreeClassifier(),
    BaggingClassifier(n_estimators=100),
    RandomForestClassifier(n_estimators=100),
    KNeighborsClassifier(n_neighbors=3),
    LogisticRegression(),
    GradientBoostingClassifier(n_estimators=100)
]

In [ ]:
results = list()
for i in range(len(names)):
    transformation_steps = [
        ('c', OneHotEncoder(handle_unknown='ignore'), cat_ix),
        ('n', MinMaxScaler(), num_ix)
    ]
    ct = ColumnTransformer(transformation_steps)
    pipeline = Pipeline(
        steps=[
            ('transform', ct),
            ('smote', SMOTE()),
            ('model', models[i])
        ])
    cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=0)
    scores = cross_val_score(pipeline, X_train, y_train, scoring='f1', cv=cv, n_jobs=-1)
    results.append(scores)
    print('>%s %.3f (%.3f)' % (names[i], np.mean(scores), np.std(scores)))
plt.boxplot(results, labels=names, showmeans=True)
plt.show()

In [ ]:
transformation_steps = [
    ('c', OneHotEncoder(handle_unknown='ignore'), cat_ix),
    ('n', MinMaxScaler(), num_ix)
]
ct = ColumnTransformer(transformation_steps)
pipeline = Pipeline(
    steps=[
        ('transform', ct),
        ('smote', SMOTE()),
        ('model', GradientBoostingClassifier())
    ])
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=0)
param_grid = {
    "model__n_estimators": [20, 100] 
}
grid_search = GridSearchCV(
    estimator=pipeline, param_grid=param_grid, cv=cv, scoring="f1_macro"
)
grid_search.fit(X_train, y_train)

In [ ]:
best_estimator = grid_search.best_estimator_

y_pred = best_estimator.predict(X_test)

from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print('ROC AUC score:', roc_auc_score(y_test, y_pred))